In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd
import numpy as np
import conda_scripts
import forward_run
import matplotlib.pyplot as plt
import conda_scripts.plot_help as ph
from conda_scripts import sv_budget
import matplotlib as mpl
import numpy as np
import flopy
import geopandas as gpd
import xarray as xr
import rioxarray as rx

In [ ]:
import matplotlib.pyplot  as plt

In [ ]:
import matplotlib as mpl

In [ ]:
mpl.rcParams["backend"] 

In [ ]:
# mpl.rcParams["backend"] = "QtAgg"

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
main = 'postprocess'
fold =  'pestaws_v1'

out_folder = os.path.join(main, fold)
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok =True)

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)

In [ ]:
import xarray as xr
import numpy as np
from pyproj import Transformer



In [ ]:
ml.update_modelgrid()

In [ ]:
xl = ml.modelgrid.get_xcellcenters_for_layer(0)
yl = ml.modelgrid.get_ycellcenters_for_layer(0)

yl

In [ ]:
shallow = rx.open_rasterio(r"C:\GSP\waterlevel\GIS\hydro_experiment\maps__v3_SON_allmodmonths_to_20240808\SON_1970_Fall_Shallow.tif")


deep = rx.open_rasterio(r"C:\GSP\waterlevel\GIS\hydro_experiment\maps__v3_SON_allmodmonths_to_20240808\SON_1970_Fall_Deep.tif")




In [ ]:
zone = ml.bas6.ibound.array[0]
zone
plt.imshow(zone)

In [ ]:
import cartopy.crs as ccrs
import contextily as ctx

In [ ]:
da = xr.DataArray(
    data=xl,
    dims=[ "y",'x'],
    coords=dict(
        x=([ "y",'x'], xl),
        y=([ "y",'x'], yl),
        # time=time,
        # reference_time=reference_time,
    ),
    attrs=dict(
        description="Ambient temperature.",
        units="degC",
    ),
)


ar_shal= shallow.isel(band = 0).interp(da.coords)
ar_deep = deep.isel(band = 0).interp(da.coords)

elev = conda_scripts.utils.elevation.load_elev_array()
elev = elev.isel(band = 0).interp(da.coords)

# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
# ar_shal.plot.contourf(ax =ax,  levels = np.arange(0, 1000, 50), alpha = .5)
# ar_shal.plot.contour(ax =ax,  levels = np.arange(0, 1000, 50))
# ax.set_title("Shallow")
# # ctx.add_basemap(ax, crs = 2226)

# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
# ar_deep.plot.contourf(ax =ax,levels = np.arange(0, 1000, 50), alpha = .5)
# ar_deep.plot.contour(ax =ax,levels = np.arange(0, 1000, 50))
# ax.set_title("Deep")
# ctx.add_basemap(ax, crs = 2226)

# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
# (ar_shal - ar_deep).plot.contour(ax =ax,levels = np.arange(-50, 51, 10))
# ax.set_title("Shallow - Deep")
# ctx.add_basemap(ax, crs = 2226)


# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
# elev.plot.contourf(ax =ax,levels = np.arange(0, 1000, 50), alpha = .5)
# elev.plot.contour(ax =ax,levels = np.arange(-50, 51, 10))
# ax.set_title("Elevation")
# ctx.add_basemap(ax, crs = 2226)

In [ ]:
import cartopy.crs as ccrs


In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12), dpi = 250, subplot_kw=dict(projection=ccrs.epsg(2226)))


levels = np.arange(-150,211, 30)

ax = ax1
# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
(elev- ar_shal).plot.contourf(ax =ax,  levels = levels, alpha = .5)
(elev- ar_shal).plot.contour(ax =ax,  levels = levels, colors = 'k', linewidths =.5)
ax.set_title('Elevation Minus Shallow GW Elevation')
# ctx.add_basemap(ax, crs = 2226)

ax = ax2
# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
(elev- ar_deep).plot.contourf(ax =ax,levels =levels, alpha = .5)
(elev- ar_deep).plot.contour(ax =ax,levels = levels, colors = 'k', linewidths =.5)
ax.set_title('Elevation Minus Deep GW Elevation')
# ctx.add_basemap(ax, crs = 2226)

ax = ax3
# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
(ar_shal- ar_deep).plot.contourf(ax =ax,levels =levels, alpha = .5)
(ar_shal- ar_deep).plot.contour(ax =ax,levels = levels, colors = 'k', linewidths =.5)
ax.set_title('Shallow Minus Deep GW Elevation')
# ctx.add_basemap(ax, crs = 2226)


ax = ax4
# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
elev.plot.contourf(ax =ax,levels = np.arange(0,1001,100),alpha = .5, cmap = 'terrain')
elev.plot.contour(ax =ax,levels = np.arange(0,1001,100), colors = 'k', linewidths =.5)
ax.set_title('Elevation')
# ctx.add_basemap(ax, crs = 2226)

In [ ]:
print(ar_shal.data.sum())
print(ar_deep.data.sum())

In [ ]:
ar_deep.data[zone==0] =-9999
ar_shal.data[zone==0] = -9999

In [ ]:
d = ar_deep.data.copy()
s = ar_shal.data.copy()

In [ ]:
(elev.data-s)<0

In [ ]:
filt = (elev.data-s)<0
assert filt.any()
s[filt] = elev.data[filt]

filt = (elev.data-d)<0
assert filt.any()
d[filt] = elev.data[filt]


In [ ]:
filt = (elev.data-s)>25
assert filt.any()
s[filt] = elev.data[filt]-25

filt = (elev.data-d)>25
assert filt.any()
d[filt] = elev.data[filt]-25

In [ ]:
print(s.sum())
print(d.sum())

In [ ]:
assert d.sum() == ar_.data.sum(), f"{d.sum()}   {ar_shal.data.sum()}"

In [ ]:
assert s.sum() == ar_shal.data.sum(), f"{s.sum()}   {ar_shal.data.sum()}"

In [ ]:
d_out = xr.DataArray(
    data=d,
    dims=[ "y",'x'],
    coords=dict(
        x=([ "y",'x'], xl),
        y=([ "y",'x'], yl),
        # time=time,
        # reference_time=reference_time,
    ),
    attrs=dict(
        description="Ambient temperature.",
        units="degC",
    ),
)

s_out = xr.DataArray(
    data=s,
    dims=[ "y",'x'],
    coords=dict(
        x=([ "y",'x'], xl),
        y=([ "y",'x'], yl),
        # time=time,
        # reference_time=reference_time,
    ),
    attrs=dict(
        description="Ambient temperature.",
        units="degC",
    ),
)

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12), dpi = 250, subplot_kw=dict(projection=ccrs.epsg(2226)))


levels = np.arange(-150,151, 10)

ax = ax1
# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
(elev- s_out).plot.contourf(ax =ax,  levels = levels, alpha = .5)
(elev- s_out).plot.contour(ax =ax,  levels = levels, colors = 'k', linewidths =.5)
ax.set_title('Elevation Minus Shallow GW Elevation')
# ctx.add_basemap(ax, crs = 2226)

ax = ax2
# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
(elev- d_out).plot.contourf(ax =ax,levels =levels, alpha = .5)
(elev- d_out).plot.contour(ax =ax,levels = levels, colors = 'k', linewidths =.5)
ax.set_title('Elevation Minus Deep GW Elevation')
# ctx.add_basemap(ax, crs = 2226)

ax = ax3
# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
(s_out- d_out).plot.contourf(ax =ax,levels =levels, alpha = .5)
(s_out- d_out).plot.contour(ax =ax,levels = levels, colors = 'k', linewidths =.5)
ax.set_title('Shallow Minus Deep GW Elevation')
# ctx.add_basemap(ax, crs = 2226)


ax = ax4
# plt.figure(dpi = 250, figsize = (6,8))
# ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
elev.plot.contourf(ax =ax,levels = np.arange(0,3000,100),alpha = .5, cmap = 'terrain')
elev.plot.contour(ax =ax,levels = np.arange(0,3000,500), colors = 'k', linewidths =.5)
ax.set_title('Elevation')
# ctx.add_basemap(ax, crs = 2226)

In [ ]:
workspace

In [ ]:
# workspace = 'pestaws_flash'

In [ ]:
for lay in range(1,7):
    if lay<3:
        print('shallow', os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"))
        np.savetxt(os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"), s_out.data, fmt = "%.0f")
    else:
        print('deep', os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"))
        np.savetxt(os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"), d_out.data, fmt = "%.0f")

In [ ]:
# for lay in range(1,7):
#     if lay<3:
#         print('shallow', os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"))
#         np.savetxt(os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"), ar_shal.data, fmt = "%.0f")
#     else:
#         print('deep', os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"))
#         np.savetxt(os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"), ar_deep.data, fmt = "%.0f")

In [ ]:
for lay in range(1,7):
    plt.figure()
    if lay<3:
        print('shallow', os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"))
        plt.imshow(ar_shal.data,vmin = 300, vmax = 800)
    else:
        print('deep', os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"))
        plt.imshow(ar_deep.data,vmin =300, vmax = 800)
        # print('deep', os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"))
        # np.savetxt(os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"), ar_deep.data, fmt = "%.0f")